<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.8-7.21/lab10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab10-1 - Convolution
===
## 목차
* Convolution이란?
* Neuran과 Convolution의 상관관계
* Pooling
* CNN implementation

# 1. Convolution이란?
이미지 위에서 ```stride```값만큼 ```filter```(kernel)을 이동시키면서 겹쳐지는 부분의 각 원소의 값을 곱해서 모두 더한 값을 출력으로 하는 연산     
![](https://ars.els-cdn.com/content/image/1-s2.0-S016971611830021X-f09-24p1-9780444640420.jpg)    
위 이미지에서 filter는,
```
filter={[1, 0, 1],
        [0, 1, 0], 
        [1, 0, 1]}
```
첫번째 stride에서
```
A = {[1, 1, 1].
     [0, 1, 1],
     [0, 0, 1]}
```

In [1]:
import torch

In [3]:
filter = torch.FloatTensor([[1, 0, 1],
                           [0, 1, 0], 
                           [1, 0, 1]])
A = torch.FloatTensor([[1, 1, 1],
                      [0, 1, 1],
                      [0, 0, 1]])

In [8]:
mul_array = filter * A
sum = mul_array.sum()
print(mul_array)
print("합:",sum)

tensor([[1., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.]])
합: tensor(4.)


나머지 부분도 마찬가지다.

## 1.1. Stride and Padding
* ```stride```: filter가 한번에 얼마나 이동할 것인지
* ```padding```: zero-padding, 이미지의 테두리를 0으로 둘러싼다.
 이미지의 상하좌우에 0으로 된 띠가 둘러진다. 만약 padding값이 2였다면, 총 두 번 둘러진다고 보면 된다.
     
![](https://blog.kakaocdn.net/dn/2hzfb/btqCJmTyg1R/unmfvdAnXwcGK16pYNAOr0/img.png)

## 1.2. Pytorch nn.Conv2d
```nn.Conv2d```를 이용해서 ```convolution layer```를 생성한다.
```
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)
```
* ```in_channel```: input value
* ```out_channel```: output value
* ```kernel_size```: filter의 크기, 3x3이면 3을 써준다. 만약 3x1을 만든다면 (3,1)을 넣어준다.
* input type: torch.Tensor
* input shape: (N x C x H x W) == (batch_size, channel, height, width)

In [9]:
import torch.nn as nn
# 예제 1
conv = nn.Conv2d(1, 1, 11, stride=4, padding=0)
print(conv)

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))


In [10]:
inputs = torch.Tensor(1, 1, 227, 227)
print(inputs.shape)

torch.Size([1, 1, 227, 227])


In [11]:
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 55, 55])


In [13]:
# 예제 2
conv = nn.Conv2d(1, 1, 7, stride=2, padding=0)
inputs = torch.Tensor(1, 1, 64, 64)
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 29, 29])


In [14]:
# 예제 3
conv = nn.Conv2d(1, 1, 5, stride=1, padding=2)
inputs = torch.Tensor(1, 1, 32, 32)
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 32, 32])


In [17]:
# 예제 4
conv = nn.Conv2d(1, 1, 5, stride=1, padding=0)
inputs = torch.Tensor(1, 1, 32, 64)
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 28, 60])


In [16]:
# 예제 5
conv = nn.Conv2d(1, 1, 3, stride=1, padding=1)
inputs = torch.Tensor(1, 1, 64, 32)
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 64, 32])


# 2. Neuran과 Convolution의 상관관계
## 2.1. Perceptron(=Neuron)과 Convolution
* Perceptron의 Weight값으로 filter가 들어간다.
* input value가 perceptron으로 들어간다.
* input value와 weight값을 곱해주고 다 더한다. 그리고 bias를 더해준다.
* 실제 output은 ```input value x weight + bias```

# 3. Pooling
이미지의 사이즈를 줄이기 위해 사용되기도 하고, Fully Connected 연산을 대체하기 위해 ```average pooling```을 사용하기도 한다.
## 3.1. Max Pooling
size가 2x2일 때, 2x2 이내에서 가장 큰 셀을 취한다. 
 
## 3.2. Average Pooling
size가 2x2일 때, 2x2 이내에서 평균을 계산해서 하나의 셀을 만들어준다.

![](https://www.researchgate.net/publication/333593451/figure/fig2/AS:765890261966848@1559613876098/Illustration-of-Max-Pooling-and-Average-Pooling-Figure-2-above-shows-an-example-of-max.png)

## 3.3. torch.nn.MaxPool2d
```
torch.nn.MaxPool2d(kernel_size, stride=None, padding=0...)
```

# 4. CNN Implementation

In [22]:
inputs = torch.Tensor(1, 1, 28, 28)         # input 1, output 1, size 28*28
conv1 = nn.Conv2d(1, 5, 5)                  # input channel 1, output channel 5, filter size 5
pool = nn.MaxPool2d(2)                      # pooling의 size 2*2
out = conv1(inputs)                         # inputs이 conv1을 거쳤을 때 
out2 = pool(out)                            # inputs이 pool을 거쳤을 때
print(out.size())
print(out2.size())

torch.Size([1, 5, 24, 24])
torch.Size([1, 5, 12, 12])


* pooling을 거치고 사이즈가 작아진 것을 확인할 수 있다.